<a href="https://colab.research.google.com/github/JENC08/dsa/blob/main/AICTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Load dataset
df = pd.read_csv('/content/test.csv')

# ✅ Step 1: Use selected columns
selected_cols = ['Gender', 'ParentalEducation', 'TestScore_Math',
                 'TestScore_Reading', 'TestScore_Science', 'GPA', 'AttendanceRate']
df = df[selected_cols].copy()

# ✅ Step 2: Create target label
df['average_score'] = df[['TestScore_Math', 'TestScore_Reading', 'TestScore_Science']].mean(axis=1)
df['Pass'] = df['average_score'].apply(lambda x: 1 if x >= 50 else 0)
df.drop(columns=['average_score'], inplace=True)

# ✅ Step 3: Encode categorical variables
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# ✅ Step 4: Train/test split and scaling
X = df.drop('Pass', axis=1)
y = df['Pass']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# ✅ Step 5: Model training with class weight
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

# ========== USER INPUT ==========

print("\n📥 Enter student details:")

def get_input(prompt, is_float=True):
    try:
        value = input(prompt)
        return float(value) if is_float else value.strip()
    except:
        print("⚠️ Invalid input. Defaulting to 0.")
        return 0.0 if is_float else ""

# Input fields
user_data = {}

# Encode Gender
gender_input = get_input(f"Gender {list(label_encoders['Gender'].classes_)}: ", is_float=False)
if gender_input in label_encoders['Gender'].classes_:
    user_data['Gender'] = label_encoders['Gender'].transform([gender_input])[0]
else:
    print("⚠️ Unknown gender. Using default.")
    user_data['Gender'] = 0

# Encode Parental Education
edu_input = get_input(f"ParentalEducation {list(label_encoders['ParentalEducation'].classes_)}: ", is_float=False)
if edu_input in label_encoders['ParentalEducation'].classes_:
    user_data['ParentalEducation'] = label_encoders['ParentalEducation'].transform([edu_input])[0]
else:
    print("⚠️ Unknown education. Using default.")
    user_data['ParentalEducation'] = 0

# Numerical Inputs
user_data['TestScore_Math'] = get_input("TestScore_Math (0-100): ")
user_data['TestScore_Reading'] = get_input("TestScore_Reading (0-100): ")
user_data['TestScore_Science'] = get_input("TestScore_Science (0-100): ")
user_data['GPA'] = get_input("GPA (0-10): ")
user_data['AttendanceRate'] = get_input("Attendance Rate (0-100): ")

# Convert to DataFrame and scale
input_df = pd.DataFrame([user_data])
input_scaled = scaler.transform(input_df)

# Predict
prediction = model.predict(input_scaled)[0]
proba = model.predict_proba(input_scaled)[0][prediction] * 100

# Show result
result = "✅ Passed" if prediction == 1 else "❌ Failed"
print(f"\n🎓 Prediction: {result} (Confidence: {proba:.2f}%)")



📥 Enter student details:
Gender ['Female', 'Male']: malw
⚠️ Unknown gender. Using default.
ParentalEducation ['<HS', 'Bachelors+', 'HS', 'SomeCollege']: degree
⚠️ Unknown education. Using default.
TestScore_Math (0-100): 25
TestScore_Reading (0-100): 21
TestScore_Science (0-100): 14
GPA (0-10): 5
Attendance Rate (0-100): 50

🎓 Prediction: ✅ Passed (Confidence: 78.00%)
